This notebook is to deal with training the model to predict the race, age, and gender from the input human face input.

The dataset source is here: https://www.kaggle.com/datasets/nipunarora8/age-gender-and-ethnicity-face-data-csv

Import the libraries here

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [2]:
import keras
import json
import sys
import tensorflow as tf
from keras.layers import Input
import argparse
from keras.utils.data_utils import get_file

In [3]:
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Dropout, Dense, Lambda
from keras.layers import Input, Flatten
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

The dataset is a csv file
Each row is defined as age ethnicity, gender, img_name, and the matrix of pixels

In [4]:
df = pd.read_csv("age_gender.csv")
df.head()
print(len(df))

FileNotFoundError: [Errno 2] No such file or directory: 'age_gender.csv'

Here we display one image to see what's stored in the dataset.

In [ ]:
df_data= pd.DataFrame(df)
df_data['pixels'] = df_data.pixels.apply(lambda x: np.reshape(np.array(x.split(' '),dtype='float32'),(48,48)))
im = df_data['pixels'][30]
plt.imshow(im, cmap='gray')
plt.axis('off')

Get the three input data columns => gender, race, and age
And also, display the unique lables of each one
To be clear, this is where we do normalization place.

In [ ]:
gender = df_data['gender']
race = df_data['ethnicity']
age = df_data['age']
images = df_data['pixels']/255 # do normalization
images = list(images)
resized_images = []
for i in images:
    temp = np.resize(i, (224,224,1))
    resized_images.append(temp)
images = np.array(list(images))
resized_images = np.array(resized_images)

print("original images shape: ", images.shape)
print("resized images shape:", resized_images.shape)

In [ ]:
display(gender.unique())
display(race.unique())
display(age.unique())

gender_num = gender.unique().shape[0]
race_num = race.unique().shape[0]
age_num = age.unique().shape[0]

In [ ]:
gender_array = gender.to_numpy()
race_array = race.to_numpy()
age_array = age.to_numpy()
max_age = np.max(age_array)
min_age = np.min(age_array)
print(gender_array.shape)
print(race_array.shape)
print(age_array.shape)

Separate the index of input dataset in train/valid/test split by 70:10:20

In [ ]:
random_indexes = np.random.permutation(len(df_data))
train_up_to = int(len(df_data) * 0.7)
valid_up_to = train_up_to + int(len(df_data) * 0.1)

train_indexes = random_indexes[:train_up_to]
valid_indexes = random_indexes[train_up_to:valid_up_to]
test_indexes = random_indexes[valid_up_to:]

print("Length of train, valid, test set indexes are {}, {}, {}".format(len(train_indexes), len(valid_indexes), len(test_indexes)))

Define a image generator

In [ ]:
def data_generator(list_index, check, batch_size, gender_array, race_array, age_array, resized_images):
    images, ages, races, genders = [], [], [], []
    while True:
        for idx in list_index:
            gender = gender_array[idx]
            age = age_array[idx]
            race = race_array[idx]
            image = resized_images[idx]

            genders.append(to_categorical(gender, gender_num))
            ages.append(age/max_age)
            races.append(to_categorical(race, race_num))
            images.append(image)

            # yielding condition
            if len(images) >= batch_size:
                yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
                images, ages, races, genders = [], [], [], []
        if not check:
            break

In [ ]:
class MultiOutputModel():
    """
    Used to generate our multi-output model. This CNN contains three branches, one for age, other for
    sex and another for race. Each branch contains a sequence of Convolutional Layers that is defined
    on the make_default_hidden_layers method.
    """
    def make_default_hidden_layers(self, inputs):
        """
        Used to generate a default set of hidden layers. The structure used in this network is defined as:

        Conv2D -> BatchNormalization -> Pooling -> Dropout
        """
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        return x
    def build_race_branch(self, inputs, num_races):
        """
        Used to build the race branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_races)(x)
        x = Activation("softmax", name="race_output")(x)
        return x
    def build_gender_branch(self, inputs, num_genders=2):
        """
        Used to build the gender branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_genders)(x)
        x = Activation("sigmoid", name="gender_output")(x)
        return x
    def build_age_branch(self, inputs):
        """
        Used to build the age branch of our face recognition network.
        This branch is composed of three Conv -> BN -> Pool -> Dropout blocks,
        followed by the Dense output layer.
        """
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation("linear", name="age_output")(x)
        return x
    def assemble_full_model(self, width, height, num_races):
        """
        Used to assemble our multi-output model CNN.
        """
        input_shape = (height, width, 1)
        inputs = Input(shape=input_shape)
        age_branch = self.build_age_branch(inputs)
        race_branch = self.build_race_branch(inputs, num_races)
        gender_branch = self.build_gender_branch(inputs)
        model = Model(inputs=inputs,
                      outputs = [age_branch, race_branch, gender_branch],
                      name="face_net")
        return model


In [ ]:
model = MultiOutputModel().assemble_full_model(224, 224, num_races=race_num)

Here is to display the structure of the model

In [ ]:
tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

Define the optimizers, learning rate, and loss function of the model

In [ ]:
from tensorflow.keras.optimizers import Adam
init_lr = 1e-4
epochs = 100
opt = Adam(lr=init_lr, decay=init_lr / epochs)
model.compile(optimizer=opt,
              loss={
                  'age_output': 'mse',
                  'race_output': 'categorical_crossentropy',
                  'gender_output': 'binary_crossentropy'},
              loss_weights={
                  'age_output': 4.,
                  'race_output': 1.5,
                  'gender_output': 0.1},
              metrics={
                  'age_output': 'mae',
                  'race_output': 'accuracy',
                  'gender_output': 'accuracy'})

In [ ]:
batch_size = 32
valid_batch_size = 32

train_generator = data_generator(list_index=train_indexes, check=True, batch_size=32, gender_array=gender_array, race_array=race_array, age_array=age_array, resized_images=resized_images)

valid_generator = data_generator(list_index=valid_indexes, check=True, batch_size=32, gender_array=gender_array, race_array=race_array, age_array=age_array, resized_images=resized_images)

In [ ]:
callbacks = [
    ModelCheckpoint("./checkpoints", monitor='val_loss')
]
history = model.fit_generator(train_generator,
                              steps_per_epoch=len(train_indexes)//batch_size,
                              epochs=epochs,
                              callbacks=callbacks,
                              validation_data=valid_generator,
                              validation_steps=len(valid_indexes)//valid_batch_size)

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['race_output_acc'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_race_output_acc'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for race feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()